## Trim BNF Information

Trim BNF Information code file to a test file to measure different fuzzy match performance to determine the best algorithm to use.  It will pick top 150 prescribed drugs in UK from Jan 2017 to Dec 2018.

In [2]:
import pandas as pd
import numpy as np
import os
import time
import re
import operator
import math
import configparser

config_file = '../config.ini'
config = configparser.ConfigParser()
config.read(config_file)

['../config.ini']

In [3]:
bnf_code_df = pd.read_csv(os.path.join(config['DEFAULT']['data_dir'],'bnf_code_clean.csv'), dtype='U')
bnf_code_df.head()

,bnf_chapter,bnf_chemical_substance,bnf_name,bnf_code,drug,form,dose,misc1
0,Gastro-Intestinal System,Other Antacid & Simeticone Preps,Langdales_Cinnamon Tab,010101000BBABA0,Langdales,Cinnamon Tab,NaN,NaN
1,Gastro-Intestinal System,Other Antacid & Simeticone Preps,Mylanta 11_Tab,010101000BBADA0,Mylanta 11,Tab,NaN,NaN
2,Gastro-Intestinal System,Other Antacid & Simeticone Preps,Mylanta 11_Liq,010101000BBAEA0,Mylanta 11,Liq,NaN,NaN
3,Gastro-Intestinal System,Other Antacid & Simeticone Preps,Rennie Plus_Tab,010101000BBAFA0,Rennie Plus,Tab,NaN,NaN
4,Gastro-Intestinal System,Other Antacid & Simeticone Preps,Sab Simplex_Susp,010101000BBAIA0,Sab Simplex,Susp,NaN,NaN


In [4]:
count_df = pd.read_csv(os.path.join(config['DEFAULT']['data_dir'], 'full_bnf_prescription_count.csv'))
count_df.head(10)

,bnf_code_full,total_prescribed
0,0103050P0AAAAAA,54264336
1,0209000A0AAABAB,35173629
2,0206020A0AAAAAA,34561560
3,0212000B0AAABAB,33173866
4,0407010H0AAAMAM,32761419
5,0103050L0AAAAAA,30506160
6,0212000Y0AAADAD,28089358
7,0601022B0AAABAB,26978271
8,0301011R0AAAPAP,24544253
9,0212000B0AAACAC,22845701


In [21]:
merge_df = bnf_code_df.merge(count_df, how='left', left_on='bnf_code', right_on='bnf_code_full')
merge_df.dropna(subset=['total_prescribed'], axis=0, inplace=True)
merge_df.drop(columns=['misc1','bnf_code_full'], inplace=True)
merge_df.sort_values(by='total_prescribed', ascending=False, inplace=True)
merge_df.head()

,bnf_chapter,bnf_chemical_substance,bnf_name,bnf_code,drug,form,dose,total_prescribed
1059,Gastro-Intestinal System,Omeprazole,Omeprazole_Cap E/C 20mg,0103050P0AAAAAA,Omeprazole,Cap E/C,20mg,54264336.0
6179,Cardiovascular System,Aspirin,Aspirin Disper_Tab 75mg,0209000A0AAABAB,Aspirin Disper,Tab,75mg,35173629.0
5119,Cardiovascular System,Amlodipine,Amlodipine_Tab 5mg,0206020A0AAAAAA,Amlodipine,Tab,5mg,34561560.0
6703,Cardiovascular System,Atorvastatin,Atorvastatin_Tab 20mg,0212000B0AAABAB,Atorvastatin,Tab,20mg,33173866.0
12396,Central Nervous System,Paracetamol,Paracet_Tab 500mg,0407010H0AAAMAM,Paracet,Tab,500mg,32761419.0


In [26]:
#slim_df = merge_df.drop_duplicates(subset=['bnf_chemical_substance'],keep='first')
final_df = slim_df.head(150)

In [23]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 1059 to 22972
Data columns (total 8 columns):
bnf_chapter               150 non-null object
bnf_chemical_substance    150 non-null object
bnf_name                  150 non-null object
bnf_code                  150 non-null object
drug                      150 non-null object
form                      149 non-null object
dose                      143 non-null object
total_prescribed          150 non-null float64
dtypes: float64(1), object(7)
memory usage: 10.5+ KB


In [27]:
final_df.to_csv(os.path.join(config['DEFAULT']['output_dir'],'bnf_code_test_file.csv'), index=False)